# Hww_Hypo




**Author:** Lailin XU  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Monday, April 19, 2021 at 09:23 AM.</small></i>

In [2]:
import ROOT as R
from math import log, sqrt, fabs

Welcome to JupyROOT 6.26/00


By szhang (Jun 6, 2022): I just find that, to import ROOT, one must activate the root environment locally! (on the terminal)

HWW simplified model: https://arxiv.org/abs/1412.2641

In [3]:
N1 = 407.
s1 = 48.6
b1 = 335.
N2, s2, b2 = 143., 16., 129.

Treat ee+mumu and emu as two separate measurements
=================

In [4]:
hs1 = N1 - b1
hs2 = N2 - b2

Delta log-likelihood

In [5]:
lam = 2*( - hs1 + N1 * log( (hs1+b1)/b1 ))
lam += 2*( - hs2 + N2 * log( (hs2+b2)/b2 ))

P-value

In [6]:
p = R.Math.chisquared_cdf_c(lam, 2.)
print(p)

0.0003458697406908297


Convert p-value to signifiance

In [7]:
nsig = R.Math.normal_quantile_c(p, 1.)

Use RooStats to convert p-value to signifiance

In [8]:
nsig_rs = R.RooStats.PValueToSignificance(p)#We could get the same result with general root---
print(nsig_rs)

3.3928323996152105


Two-sided siginifance

In [9]:
nsig2 = R.Math.normal_quantile_c(p*0.5, 1.)

print("Lambda = {0}, p-value= {1}, significance= {2} {3}".format(lam, p, nsig, nsig2))

Lambda = 15.938896651563486, p-value= 0.0003458697406908297, significance= 3.3928323996152105 3.5782747685509237


One single POI (parameter of interest)
=================
Maximize the likelihood to get the best estimate of mu

In [10]:
a = (s1 + s2)*s1*s2
b = (s1+s2)*(b1*s2 + b2*s1) - (N1+N2)*s1*s2
c = b1*b2*(s1+s2) - N1*s1*b2 - N2*s2*b1

MLE of mu

In [11]:
mu = (-b + sqrt( pow(b, 2) - 4.*a*c))/(2.*a)

print("mu = {0}".format(mu))

hs1 = mu*s1
hs2 = mu*s2

lam = 2*( - hs1 + N1 * log( (hs1+b1)/b1 ))
lam += 2*( - hs2 + N2 * log( (hs2+b2)/b2 ))

p = R.Math.chisquared_cdf_c(lam, 1.)
nsig = R.Math.normal_quantile_c(p, 1.)
nsig_rs = R.RooStats.PValueToSignificance(p)
nsig2 = R.Math.normal_quantile_c(p*0.5, 1.)

print("Lambda = {0}, p-value= {1}, significance= {2} {3}".format(lam, p, nsig, nsig2))

mu = 1.3457630433503782
Lambda = 15.44749401658941, p-value= 8.482934894864863e-05, significance= 3.7603752029871957 3.930330013699793


Calculate NLL explicitly

In [12]:
x = 0
lnL0 = ( -x* s1 - b1 + N1*log(x*s1+b1) ) + (-x*s2 - b2 + N2*log(x*s2+b2) )
print("mu={0}, NLL= {1}".format(x, lnL0))
x = mu
lnL1 = ( -x* s1 - b1 + N1*log(x*s1+b1) ) + (-x*s2 - b2 + N2*log(x*s2+b2) )
print("mu={0}, NLL= {1}".format(x, lnL1))
dNLL = 2*(lnL1-lnL0)
print("dNLL={0}, Z0= {1}".format(dNLL, sqrt(dNLL)))

mu=0, NLL= 2597.304300276521
mu=1.3457630433503782, NLL= 2605.0280472848153
dNLL=15.44749401658828, Z0= 3.9303300136996486


Use RooFit/RooStat to do the hypotest
=================
Instantiate a workspace

**From szhang (Jun 6, 2022)**: break a fly upon a wheel--- While RooFit/RooStat is of great importance!

From szhang (Jun 6, 2022): Currently not be understood and mastered---

In [13]:
w = R.RooWorkspace("w")

Create pdf components

In [14]:
w.factory("expr::s_emu('mu*n_emu',mu[1,0,10],n_emu[48.6])")
w.factory("sum:nexp_emu(s_emu,b_emu[335])")
w.factory("expr::s_ll('mu*n_ll',mu,n_ll[16])")
w.factory("sum:nexp_ll(s_ll,b_ll[129])")
w.factory("Poisson:emu(nobs_emu[0,1000],nexp_emu)")
w.factory("Poisson:ll(nobs_ll[0,1000],nexp_ll)")

Create the total model

In [15]:
w.factory("PROD:model(emu,ll)")

Data

In [16]:
r_emu = w.var("nobs_emu")
r_ll = w.var("nobs_ll")
data = R.RooDataSet("ds", "ds", R.RooArgSet(r_emu, r_ll))

r_emu.setVal(407)
r_ll.setVal(143)
data.add(R.RooArgSet(r_emu, r_ll))

w.Import(data, R.RooFit.Rename("ObsData"))

model = w.pdf("model")
model.Print()

[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing dataset ds
[#1] INFO:ObjectHandling -- RooWorkSpace::import(w) changing name of dataset from  ds to ObsData
RooProdPdf::model[ emu * ll ] = 0.000322962


Create the ModelConfig

In [17]:
mc=R.RooStats.ModelConfig("ModelConfig", w)

Set up the Model

In [18]:
mc.SetPdf(w.pdf("model"))
mc.SetParametersOfInterest(R.RooArgSet(w.var("mu")))
mc.SetObservables(R.RooArgSet(w.var("nobs_emu"), w.var("nobs_ll")))

mc.Print()

w.Import(mc)

False


=== Using the following for ModelConfig ===
Observables:             RooArgSet:: = (nobs_emu,nobs_ll)
Parameters of Interest:  RooArgSet:: = (mu)
PDF:                     RooProdPdf::model[ emu * ll ] = 0.000322962



Take a peek at the workspac

In [19]:
w.Print("t")


RooWorkspace(w) w contents

variables
---------
(b_emu,b_ll,mu,n_emu,n_ll,nobs_emu,nobs_ll)

p.d.f.s
-------
RooProdPdf::model[ emu * ll ] = 0.000322962
  RooPoisson::emu[ x=nobs_emu mean=nexp_emu ] = 0.00982155
    RooAddition::nexp_emu[ s_emu + b_emu ] = 383.6
      RooFormulaVar::s_emu[ actualVars=(mu,n_emu) formula="mu*n_emu" ] = 48.6
  RooPoisson::ll[ x=nobs_ll mean=nexp_ll ] = 0.032883
    RooAddition::nexp_ll[ s_ll + b_ll ] = 145
      RooFormulaVar::s_ll[ actualVars=(mu,n_ll) formula="mu*n_ll" ] = 16

datasets
--------
RooDataSet::ObsData(nobs_emu,nobs_ll)

named sets
----------
ModelConfig_Observables:(nobs_emu,nobs_ll)
ModelConfig_POI:(mu)

generic objects
---------------
RooStats::ModelConfig::ModelConfig



Fitting

In [20]:
model.fitTo(data)

data.Print("v")

[#1] INFO:Minimization -- createConstraintTerm: caching constraint set under name CACHE_CONSTR_OF_PDF_model_FOR_OBS_nobs_emu:nobs_ll with 0 entries
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
[#1] INFO:Minimization --  The following expressions will be evaluated in cache-and-track mode: (emu,ll)
 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 mu           1.00000e+00  5.00000e-01    0.00000e+00  1.00000e+01
 **********
 **    3 **SET ERR         0.5
 **********
 **********
 **    4 **SET PRINT           1
 **********
 **********
 **    5 **SET STR           1
 **********
 NOW USING STRATEGY  1: TRY TO BALANCE SPEED AGAINST RELIABILITY
 **********
 **    6 **MIGRAD         500           1
 **********
 FIRST CALL TO USER FUNCTION AT NEW START POINT, WITH IFLAG=4.
 START MIGRAD MINIMIZATION.  STRATEG

Asymptotic calculator
=================

**From szhang (Jun 6, 2022)** : Currently not be understood nor mastered---

The S+B model (Alternative hypo)

In [21]:
sbModel = w.obj("ModelConfig")
pois = sbModel.GetParametersOfInterest()
pois[0].setVal(1)
poi = pois.first()
sbModel.SetSnapshot(R.RooArgSet(pois))

PDF

In [22]:
pdf = sbModel.GetPdf()

save snapshot before any fit has been done

In [23]:
params = pdf.getParameters(data)
snapshotName_init = "snapshot_paramsVals_initial"
w.saveSnapshot(snapshotName_init, params)

True

The B model (Null hypo)

In [24]:
bModel = sbModel.Clone()
bModel.SetName("B_only_model")
pois[0].setVal(0)
bModel.SetSnapshot(R.RooArgSet(pois))
bModel.Print()

w.Print()


=== Using the following for B_only_model ===
Observables:             RooArgSet:: = (nobs_emu,nobs_ll)
Parameters of Interest:  RooArgSet:: = (mu)
PDF:                     RooProdPdf::model[ emu * ll ] = 2.27995e-07
Snapshot:                
  1) 0x55f6193e9a20 RooRealVar:: mu = 0 +/- 0.361779  L(0 - 10)  "mu"


RooWorkspace(w) w contents

variables
---------
(b_emu,b_ll,mu,n_emu,n_ll,nobs_emu,nobs_ll)

p.d.f.s
-------
RooPoisson::emu[ x=nobs_emu mean=nexp_emu ] = 1.42409e-05
RooPoisson::ll[ x=nobs_ll mean=nexp_ll ] = 0.0160098
RooProdPdf::model[ emu * ll ] = 2.27995e-07

functions
--------
RooAddition::nexp_emu[ s_emu + b_emu ] = 335
RooAddition::nexp_ll[ s_ll + b_ll ] = 129
RooFormulaVar::s_emu[ actualVars=(mu,n_emu) formula="mu*n_emu" ] = 0
RooFormulaVar::s_ll[ actualVars=(mu,n_ll) formula="mu*n_ll" ] = 0

datasets
--------
RooDataSet::ObsData(nobs_emu,nobs_ll)

parameter snapshots
-------------------
ModelConfig__snapshot = (mu=1 +/- 0.361779)
snapshot_paramsVals_initial = (mu=1 +

Asymptotic calculator

In [25]:
ac = R.RooStats.AsymptoticCalculator(data, sbModel, bModel)
ac.SetOneSidedDiscovery(True)

[#0] PROGRESS:Eval -- AsymptoticCalculator::Initialize....
[#0] PROGRESS:Eval -- AsymptoticCalculator::Initialize - Find  best unconditional NLL on observed data
AsymptoticCalculator::EvaluateNLL  ........ using Minuit / Migrad with strategy  1 and tolerance 1
 **********
 **   10 **SET PRINT           0
 **********
 **********
 **   11 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 mu           0.00000e+00  3.61779e-01    0.00000e+00  1.00000e+01
 MINUIT WARNING IN PARAM DEF
 ============== STARTING VALUE IS AT LIMIT.
 MINUIT WARNING IN PARAMETR
 ============== VARIABLE1 IS AT ITS LOWER ALLOWED LIMIT.
 MINUIT WARNING IN PARAMETR
 ============== VARIABLE1 BROUGHT BACK INSIDE LIMITS.
 **********
 **   12 **SET ERR         0.5
 **********
 **********
 **   13 **SET PRINT           0
 **********
 **********
 **   14 **SET STR           1
 **********
 **********
 **   15 **MIGRAD         500           1
 **********
 MINUIT WA

Get the hypo test result

In [26]:
asResult = ac.GetHypoTest()
asResult.Print()
pvalue_as = asResult.NullPValue()


[#1] INFO:Eval -- AsymptoticCalculator::GetHypoTest: - perform  an hypothesis test for  POI ( mu ) = 0
[#0] PROGRESS:Eval -- AsymptoticCalculator::GetHypoTest -  Find  best conditional NLL on OBSERVED data set ..... 
AsymptoticCalculator::EvaluateNLL -  value = 15.2939 for poi fixed at = 0
[#0] PROGRESS:Eval -- 	 OBSERVED DATA :  qmu   = 15.4475 condNLL = 15.2939 uncond 7.5702
[#0] PROGRESS:Eval -- AsymptoticCalculator::GetHypoTest -- Find  best conditional NLL on ASIMOV data set .... 
AsymptoticCalculator::EvaluateNLL -  value = 11.6216 for poi fixed at = 0
[#0] PROGRESS:Eval -- 	 ASIMOV data qmu_A = 8.63963 condNLL = 11.6216 uncond 7.30184
[#0] PROGRESS:Eval -- poi = 0 qmu = 15.4475 qmu_A = 8.63963 sigma = 0  CLsplusb = 4.24149e-05 CLb = 0.160842 CLs = 3792.1

Results HypoTestAsymptotic_result: 
 - Null p-value = 4.24149e-05
 - Significance = 3.93033
 - CL_b: 4.24149e-05
 - CL_s+b: 0.160842
 - CL_s: 3792.1


By hand calculation
=======================

**From szhang (Jun 6, 2022)** : Currently not be understood nor mastered---

In [27]:
w.loadSnapshot(snapshotName_init)
sbModel = w.obj("ModelConfig")
pdf = sbModel.GetPdf()

Get the nuisance parameters and global observables

In [28]:
constrainedParams = sbModel.GetNuisanceParameters()
glbObs = sbModel.GetGlobalObservables()

Create the neg-log-likelihood

In [ ]:
nll_sb = pdf.createNLL(data, R.RooFit.Constrain(constrainedParams), R.RooFit.GlobalObservables(glbObs),
                                       R.RooFit.NumCPU(2), R.RooFit.Optimize(2))
nllval = nll_sb.getVal()
print("Starting NLL value:", nllval)

Do the minimization

In [1]:
minim = R.RooMinimizer(nll_sb)
strategy = R.Math.MinimizerOptions.DefaultStrategy()
minim.setStrategy(strategy)
minim.optimizeConst(2)
minimizer = R.Math.MinimizerOptions.DefaultMinimizerType()
algorithm = R.Math.MinimizerOptions.DefaultMinimizerAlgo()
print("\n =========== Unconditinal fit =========\n")
status = minim.minimize(minimizer, algorithm)

obs_nll_min = nll_sb.getVal()
reverse = (poi.getVal() < 0)

NameError: name 'R' is not defined

Fix POI to 0 (B-only model) and do the minimization again

In [30]:
print("\n =========== Conditinal fit =========\n")
w.loadSnapshot(snapshotName_init)
poi.setVal(0)
poi.setConstant(1)

status = minim.minimize(minimizer, algorithm)

obs_nll_min_bkg = nll_sb.getVal()


 =========== Conditinal fit =========

[#1] INFO:Minization -- RooMinimizerFcn::synchronize: set of constant parameters changed, rerunning const optimizer
 **********
 **   22 **SET PRINT           1
 **********
 **********
 **   23 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 mu           0.00000e+00  1.00000e-01     no limits
 **********
 **   24 **FIX           1
 **********


Info in <TMinuitMinimizer::Minimize>: There are no free parameter - just compute the function value


The asymptotic statistic: q0 = nll(b-only, mu=0) - nll(s+b)
Significance: Z = sqrt( 2*q0 )

In [31]:
obs_q0 = 2*(obs_nll_min_bkg - obs_nll_min)

Check the sign: excess or deficit? 

In [32]:
if reverse: obs_q0 = -obs_q0
sign = 0
if obs_q0!=0: sign = obs_q0 / fabs(obs_q0)
obs_sig = sign*sqrt(fabs(obs_q0));
print("\nUnconditional NLL value:", obs_nll_min)
print("Conditional NLL value:", obs_nll_min_bkg)
print("dNLL={0}, Z0= {1}".format(obs_q0, sqrt(obs_q0)))
print("==> Asymmptotic signficance: ", obs_sig)


Unconditional NLL value: 7.570194792128092
Conditional NLL value: 15.293941799838876
dNLL=15.447494015421569, Z0= 3.9303300135512247
==> Asymmptotic signficance:  3.9303300135512247


Draw all canvases 

In [33]:
from ROOT import gROOT 
gROOT.GetListOfCanvases().Draw()